In [1]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player2_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to Player 2.
    """
    visited = set()
    queue = deque(player2_cells)
    for cell in player2_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 9,
    m: int = 3,
    n: int = 3
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_unreachable_cells)
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Unreachable cells are empty cells not in 'reachable'
                # Exclude the cell just filled by Player 1
                remaining_empty = all_empty - {cell}
                unreachable = remaining_empty - reachable
                
                if unreachable:
                    # Found a move that cordons off some cells
                    cordoned_moves.append((cell, sorted(list(unreachable))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    return cordoned_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nCompetitive Sudoku Grid:")
    print("\n    " + "  ".join(str(i) for i in range(len(grid))))
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nGrid with Unreachable Cells Highlighted:")
    print("\n    " + "  ".join(str(i) for i in range(len(grid))))
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append("X")
            else:
                row_display.append(legend.get(cell, str(cell)))
        print(f"{r}  " + "  ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup
    # 0 = empty, 1 = Player 1, 2 = Player 2
    player1 = [(1, 1), (1, 0), (2, 1), (3, 1), (4, 1)]
    player2 = [(7, 8), (8, 7), (8, 8)]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(9)] for _ in range(9)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to make the following cells unreachable to Player 2:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")


Initial Grid Setup:

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8
0  .  .  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  .  .  .  .  .  .  .
6  .  .  .  .  .  .  .  .  .
7  .  .  .  .  .  .  .  .  P2
8  .  .  .  .  .  .  .  P2  P2

Player 1 can fill cell (0, 1) to make the following cells unreachable to Player 2:
[(0, 0)]

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8
0  .  P1  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  .  .  .  .  .  .  .
6  .  .  .  .  .  .  .  .  .
7  .  .  .  .  .  .  .  .  P2
8  .  .  .  .  .  .  .  P2  P2


Grid with Unreachable Cells Highlighted:

    0  1  2  3  4  5  6  7  8
0  X  P1  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  

In [2]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player2_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to Player 2.
    """
    visited = set()
    queue = deque(player2_cells)
    for cell in player2_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 9,
    m: int = 3,
    n: int = 3
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_unreachable_cells)
    Only returns moves that block the maximum number of cells.
    If multiple moves block the same maximum number, all are returned.
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Unreachable cells are empty cells not in 'reachable'
                # Exclude the cell just filled by Player 1
                remaining_empty = all_empty - {cell}
                unreachable = remaining_empty - reachable
                
                if unreachable:
                    # Found a move that cordons off some cells
                    cordoned_moves.append((cell, sorted(list(unreachable))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    if not cordoned_moves:
        return []
    
    # Determine the maximum number of blocked cells
    max_blocked = max(len(unreachable) for _, unreachable in cordoned_moves)
    
    # Filter moves that have the maximum number of blocked cells
    best_moves = [
        (cell, unreachable)
        for cell, unreachable in cordoned_moves
        if len(unreachable) == max_blocked
    ]
    
    return best_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nCompetitive Sudoku Grid:\n")
    print("    " + "  ".join(str(i) for i in range(len(grid))))
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nGrid with Unreachable Cells Highlighted:\n")
    print("    " + "  ".join(str(i) for i in range(len(grid))))
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append("X")
            else:
                row_display.append(legend.get(cell, str(cell)))
        print(f"{r}  " + "  ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup
    # 0 = empty, 1 = Player 1, 2 = Player 2
#     player1 = [(1, 1), (2, 1), (2, 0), (3, 1), (4, 1), (5, 1)]
#     player2 = [(7, 8), (8, 7), (8, 8)]
    
#     # Initialize grid for visualization
#     initial_grid = [[0 for _ in range(9)] for _ in range(9)]
#     for (r, c) in player1:
#         initial_grid[r][c] = 1
#     for (r, c) in player2:
#         initial_grid[r][c] = 2


    # Player 1 occupies cells forming a partial barrier around an internal area
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 # (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        # Determine the maximum number of blocked cells
        max_blocked = len(cordoned_scenarios[0][1])
        print(f"Maximum number of cells blocked: {max_blocked}\n")
        
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to block {len(unreachable)} cells:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")


Initial Grid Setup:

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
3  .  .  P2  .  .  .  .  .  .  .  .  .  .  .  .  .
4  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
5  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
6  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
7  .  .  .  .  .  P1  .  .  .  .  .  .  .  .  .  .
8  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
9  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
11  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
12  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
13  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
14  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
15  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

Maximum number of cells blocked: 5

Player 1 can fill cell (6, 4) to block

In [3]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to the given player.
    """
    visited = set()
    queue = deque(player_cells)
    for cell in player_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 16,
    m: int = 4,
    n: int = 4
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_newly_unreachable_cells)
    Only returns moves that block the maximum number of additional cells.
    If multiple moves block the same maximum number, all are returned.
    If no moves can block any cells, returns an empty list.
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Initial reachable cells for Player 2
    initial_reachable = get_reachable_cells(grid, player2_cells, N)
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # If no adjacent empty cells, no possible moves to block
    if not adjacent_empty_cells:
        return []
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                new_reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Newly blocked cells are those that were reachable before but not after
                newly_blocked = initial_reachable - new_reachable
                
                if newly_blocked:
                    # Found a move that blocks some cells
                    cordoned_moves.append((cell, sorted(list(newly_blocked))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    if not cordoned_moves:
        return []
    
    # Determine the maximum number of newly blocked cells
    max_blocked = max(len(unreachable) for _, unreachable in cordoned_moves)
    
    # Filter moves that have the maximum number of blocked cells
    best_moves = [
        (cell, unreachable)
        for cell, unreachable in cordoned_moves
        if len(unreachable) == max_blocked
    ]
    
    return best_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    # Header with column numbers
    header = "     " + "  ".join(f"{i:2}" for i in range(len(grid)))
    print("\nCompetitive Sudoku Grid:\n")
    print(header)
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx:2}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    # Header with column numbers
    header = "     " + "  ".join(f"{i:2}" for i in range(len(grid)))
    print("\nGrid with Unreachable Cells Highlighted:\n")
    print(header)
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append(" X")
            else:
                display = legend.get(cell, str(cell))
                # Adjust spacing for single-digit numbers
                if isinstance(display, int) and display < 10:
                    display = f" {display}"
                row_display.append(display)
        print(f"{r:2}  " + " ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup for a 16x16 grid
    # 0 = empty, 1 = Player 1, 2 = Player 2
    # We'll create a scenario where Player 1 can block an internal area
    # Additionally, we'll create a test case where the grid is already fully blocked

    # Incorrectly states blocked off scenario, because area is already blocked off    
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        # Determine the maximum number of blocked cells
        max_blocked = len(cordoned_scenarios[0][1])
        print(f"Maximum number of cells blocked: {max_blocked}\n")
        
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to block {len(unreachable)} cells:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")

Initial Grid Setup:

Competitive Sudoku Grid:

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
 3  .  .  P2  .  .  .  .  .  .  .  .  .  .  .  .  .
 4  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 5  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
 6  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
 7  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
 8  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
 9  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
11  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
12  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
13  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
14  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
15  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

Maximum number of cells blocked: 1

Player 1 can fil

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  . . . . . . . . . . . . . . . .
 1  . . . . . . . . . . . . . . . .
 2  . . P2 P2 P2 . . . . . . . . . . .
 3  . . P2 . . . . . . . . . . . . .
 4  . . . . . . . . . . . . . . . .
 5  . . . . . P1 P1 P1 P1 . . . . . . .
 6  . . . . . P1 . . P1 . . . . . . .
 7  . . . . . P1 . . P1 . . . . . . .
 8  . . . . . P1 P1 P1 P1 . . . . . . .
 9  . . . . . . .  X . . . . . . . .
10  . . . . . . . . . . . . . . . .
11  . . . . . . . . . . . . . . . .
12  . . . . . . . . . . . . . . . .
13  . . . . . . . . . . . . . . . .
14  . . . . . . . . . . . . . . . .
15  . . . . . . . . . . . . . . . .

Player 1 can fill cell (6, 4) to block 1 cells:
[(6, 4)]

Competitive Sudoku Grid:

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
 3  .  .  P2  .  .  .  .  .  .

In [4]:
# Two equal sized blocked off areas can be created, return both    
    
    player1 = [(1, 1), (1, 0), (2, 1), (3, 1), (4, 1)]
    player2 = [(7, 8), (8, 7), (8, 8)]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(9)] for _ in range(9)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2

IndentationError: unexpected indent (<ipython-input-4-b6a2dd1e8ac8>, line 3)

In [ ]:
# Incorrectly states blocked off scenario, because area is already blocked off    
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2

In [ ]:
# Example not blocked off because opponents cell in the 'wall"

    player1 = [
#         (5, 5), 
                (5, 6), (5, 7), (5, 8),
        (6, 5),           (6, 8),
        (7, 5),           (7, 8),
        (8, 5),        (8, 6), (8, 7), (8, 8)
    ]
    

    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2), (8, 6)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2

In [59]:
from collections import deque

def find_path_with_max_one_empty_cell(node):
    """
    Find a path from the left wall (excluding row 0 and excluding (0, 0)) to either
    the upper wall or the right wall that minimizes the number of empty cells on the path
    and ensures at most one empty cell is included.

    Parameters:
    - node: The GameState object.

    Returns:
    - A list of tuples representing the empty cells on the path.
    """
    N = node.board.N  # Board size (N x N grid)
    current_player_occupied = (
        node.occupied_squares1 if node.my_player == 1 else node.occupied_squares2
    )

    def is_valid_cell(row, col, visited, empty_cell_count):
        """
        Check if a cell is valid for inclusion in the path.
        - It must be within bounds.
        - It must not be (0, 0).
        - It must be empty or occupied by the current player.
        - It must not have been visited before.
        - It must not exceed the maximum empty cell limit.
        """
        return (
                (row, col) != (0, 0) and  # Exclude (0, 0)
                0 <= row < N and 0 <= col < N and
                (row, col) not in visited and
                (
                        (node.board.get((row, col)) == 0 and empty_cell_count < 1) or
                        (row, col) in current_player_occupied
                )
        )

    def bfs():
        """
        Perform BFS to find a path from the left wall (excluding row 0 and (0, 0))
        to the upper or right wall.
        """
        # Initialize queue with starting points on the left wall (excluding row 0 and (0, 0))
        queue = deque([
            (row, 0, [], 0)  # (row, col, path, empty_cell_count)
            for row in range(1, N)
            if node.board.get((row, 0)) == 0 or (row, 0) in current_player_occupied
        ])
        visited = set()
        best_path = []
        min_empty_cells = float('inf')

        while queue:
            row, col, path, empty_cell_count = queue.popleft()

            if (row, col) in visited:
                continue
            visited.add((row, col))

            # Add the current cell to the path
            new_path = path + [(row, col)]
            new_empty_cell_count = empty_cell_count + (1 if node.board.get((row, col)) == 0 else 0)

            # Stop exploring if empty cell count exceeds the limit
            if new_empty_cell_count > 1:
                continue

            # Check if the path reaches the upper wall or the right wall
            if row == 0 and col != 0 or col == N - 1: 
                #print(new_path)# Exclude (0, 0) explicitly
                if new_empty_cell_count < min_empty_cells:
                    min_empty_cells = new_empty_cell_count
                    best_path = new_path
                elif new_empty_cell_count == min_empty_cells:
                    # Break ties by prioritizing shorter paths
                    if not best_path or len(new_path) < len(best_path):
                        best_path = new_path

            # Explore neighbors (horizontal and vertical only)
            neighbors = [
                (row - 1, col), (row + 1, col),  # Vertical
                (row, col - 1), (row, col + 1)   # Horizontal
            ]

            # Filter neighbors to include only valid board coordinates
            neighbors = [
                (nr, nc) for nr, nc in neighbors
                if 0 <= nr < N and 0 <= nc < N  # Ensure within bounds
            ]

            neighbors.sort(key=lambda x: node.board.get(x) == 0)  # Prefer player-occupied cells

            for nr, nc in neighbors:
                if is_valid_cell(nr, nc, visited, new_empty_cell_count):
                    queue.append((nr, nc, new_path, new_empty_cell_count))

        return best_path

    # Perform the BFS to find the best path
    best_path = bfs()

    # Extract and return the empty cells on the best path
    return [cell for cell in best_path if node.board.get(cell) == 0]

from collections import deque

def find_path_from_right_wall(node):
    """
    Find a path from the right wall to either the upper wall or the left wall
    that minimizes the number of empty cells on the path and ensures at most one empty cell.

    Parameters:
    - node: The GameState object.

    Returns:
    - A list of tuples representing the empty cells on the path.
    """
    N = node.board.N  # Board size (N x N grid)
    current_player_occupied = (
        node.occupied_squares1 if node.my_player == 1 else node.occupied_squares2
    )

    def is_valid_cell(row, col, visited, empty_cell_count):
        """
        Check if a cell is valid for inclusion in the path.
        - It must be within bounds.
        - It must not be (0, 0).
        - It must be empty or occupied by the current player.
        - It must not have been visited before.
        - It must not exceed the maximum empty cell limit.
        """
        return (
                (row, col) != (0, N - 1) and  # Exclude (0, N - 1)
                0 <= row < N and 0 <= col < N and
                (row, col) not in visited and
                (
                        (node.board.get((row, col)) == 0 and empty_cell_count < 1) or
                        (row, col) in current_player_occupied
                )
        )

    def bfs():
        """
        Perform BFS to find a path from the right wall to the upper or left wall.
        """
        # Initialize queue with starting points on the right wall
        queue = deque([
            (row, N - 1, [], 0)  # (row, col, path, empty_cell_count)
            for row in range(N)
            if node.board.get((row, N - 1)) == 0 or (row, N - 1) in current_player_occupied
        ])
        visited = set()
        best_path = []
        min_empty_cells = float('inf')

        while queue:
            row, col, path, empty_cell_count = queue.popleft()

            if (row, col) in visited:
                continue
            visited.add((row, col))

            # Add the current cell to the path
            new_path = path + [(row, col)]
            new_empty_cell_count = empty_cell_count + (1 if node.board.get((row, col)) == 0 else 0)

            # Stop exploring if empty cell count exceeds the limit
            if new_empty_cell_count > 1:
                continue

            # Check if the path reaches the upper wall or the left wall
            if (row == 0 and col != N - 1) or (col == 0 and row != 0):  # Exclude (0, N - 1) explicitly
                if new_empty_cell_count < min_empty_cells:
                    min_empty_cells = new_empty_cell_count
                    best_path = new_path
                elif new_empty_cell_count == min_empty_cells:
                    # Break ties by prioritizing shorter paths
                    if not best_path or len(new_path) < len(best_path):
                        best_path = new_path

            # Explore neighbors (horizontal and vertical only)
            neighbors = [
                (row - 1, col), (row + 1, col),  # Vertical
                (row, col - 1), (row, col + 1)   # Horizontal
            ]

            # Filter neighbors to include only valid board coordinates
            neighbors = [
                (nr, nc) for nr, nc in neighbors
                if 0 <= nr < N and 0 <= nc < N  # Ensure within bounds
            ]

            neighbors.sort(key=lambda x: node.board.get(x) == 0)  # Prefer player-occupied cells

            for nr, nc in neighbors:
                if is_valid_cell(nr, nc, visited, new_empty_cell_count):
                    queue.append((nr, nc, new_path, new_empty_cell_count))

        return best_path

    # Perform the BFS to find the best path
    best_path = bfs()

    # Extract and return the empty cells on the best path
    return [cell for cell in best_path if node.board.get(cell) == 0]



In [224]:
from collections import deque

def find_path_player_left(node):
    """
    Find a path that minimizes the number of empty cells and ensures at most one empty cell.
    The path can start at the left wall (excluding row 0 and (0, 0)) if Player 1, or at the right wall if Player 2.
    The path can end at the upper wall, the right wall, or the left wall based on the player's position.

    Parameters:
    - node: The GameState object.

    Returns:
    - A list of tuples representing the empty cells on the path.
    """
    N = node.board.N  # Board size (N x N grid)
    current_player_occupied = (
        node.occupied_squares1 if node.my_player == 1 else node.occupied_squares2
    )

    def is_valid_cell(row, col, visited, empty_cell_count):
        """
        Check if a cell is valid for inclusion in the path.
        - It must be within bounds.
        - It must not be a restricted starting cell like (0, 0) or (N-1, N-1).
        - It must be empty or occupied by the current player.
        - It must not have been visited before.
        - It must not exceed the maximum empty cell limit.
        """
        return (
                (row, col) != (0, 0) and (row, col) != (N - 1, 0) and
                0 <= row < N and 0 <= col < N and
                (row, col) not in visited and
                (
                        (node.board.get((row, col)) == 0 and empty_cell_count < 1) or
                        (row, col) in current_player_occupied
                )
        )

    def bfs():
        """
        Perform BFS to find a path based on the current player's perspective.
        """
        # Determine starting and ending walls based on the player
        if node.my_player == 1:
            # Player 1 starts from the left wall (excluding row 0)
            starting_points = [
                (row, 0, [], 0, (row, 0))  # (row, col, path, empty_cell_count)
                for row in range(1, N)
                if node.board.get((row, 0)) == 0 or (row, 0) in current_player_occupied
            ]
            ending_conditions = lambda r, c: (r == 0 and c != 0) or (c == N - 1)  # Ends at upper or right wall
        else:
            # Player 2 starts from the right wall (excluding row N-1)
            starting_points = [
                (row, 0, [], 0, (row, 0))  # (row, col, path, empty_cell_count)
                for row in range(N - 1)
                if node.board.get((row, 0)) == 0 or (row, 0) in current_player_occupied
            ]
            ending_conditions = lambda r, c: (r == N-1 and c != 0) or (c == N-1)  # Ends at upper or left wall

        # Initialize the queue
        queue = deque(starting_points)
        visited = set()
        best_path = []
        min_empty_cells = float('inf')
        all_valid_paths = []
        while queue:
            row, col, path, empty_cell_count, origin = queue.popleft()

            if (row, col, origin) in visited:
                continue
            visited.add((row, col, origin))

            # Add the current cell to the path
            new_path = path + [(row, col)]
            new_empty_cell_count = empty_cell_count + (1 if node.board.get((row, col)) == 0 else 0)

            # Stop exploring if empty cell count exceeds the limit
            if new_empty_cell_count > 1:
                continue

            # Check if the path satisfies the ending conditions
            if ending_conditions(row, col) and new_empty_cell_count == 1:
                all_valid_paths.append(new_path)

            # Explore neighbors (horizontal and vertical only)
            neighbors = [
                (row - 1, col), (row + 1, col),  # Vertical
                (row, col - 1), (row, col + 1)   # Horizontal
            ]

            # Filter neighbors to include only valid board coordinates
            neighbors = [
                (nr, nc) for nr, nc in neighbors
                if 0 <= nr < N and 0 <= nc < N  # Ensure within bounds
            ]

            neighbors.sort(key=lambda x: node.board.get(x) == 0)  # Prefer player-occupied cells

            for nr, nc in neighbors:
                if is_valid_cell(nr, nc, visited, new_empty_cell_count):
                    queue.append((nr, nc, new_path, new_empty_cell_count, origin))

        return all_valid_paths

    # Perform the BFS to find the best path
    paths = bfs()
    # Extract and return the empty cells on the best path
    return paths#[cell for path in paths for cell in path if node.board.get(cell) == 0]



def find_path_player_right(node):
    """
    Find a path that minimizes the number of empty cells and ensures at most one empty cell.
    The path can start at the left wall (excluding row 0 and (0, 0)) if Player 1, or at the right wall if Player 2.
    The path can end at the upper wall, the right wall, or the left wall based on the player's position.

    Parameters:
    - node: The GameState object.

    Returns:
    - A list of tuples representing the empty cells on the path.
    """
    N = node.board.N  # Board size (N x N grid)
    current_player_occupied = (
        node.occupied_squares1 if node.my_player == 1 else node.occupied_squares2
    )

    def is_valid_cell(row, col, visited, empty_cell_count):
        """
        Check if a cell is valid for inclusion in the path.
        - It must be within bounds.
        - It must not be a restricted starting cell like (0, 0) or (N-1, N-1).
        - It must be empty or occupied by the current player.
        - It must not have been visited before.
        - It must not exceed the maximum empty cell limit.
        """
        return (
                (row, col) != (0, N-1) and (row, col) != (N - 1, N-1) and
                0 <= row < N and 0 <= col < N and
                (row, col) not in visited and
                (
                        (node.board.get((row, col)) == 0 and empty_cell_count < 1) or
                        (row, col) in current_player_occupied
                )
        )

    def bfs():
        """
        Perform BFS to find a path based on the current player's perspective.
        """
        # Determine starting and ending walls based on the player
        if node.my_player == 1:
            # Player 1 starts from the left wall (excluding row 0)
            starting_points = [
                (row, N-1, [], 0, (row, N-1))  # (row, col, path, empty_cell_count)
                for row in range(1, N)
                if node.board.get((row, N-1)) == 0 or (row, N-1) in current_player_occupied
            ]
            ending_conditions = lambda r, c: (r == 0 and c != N-1)   # Ends at upper or right wall
        else:
            # Player 2 starts from the right wall (excluding row N-1)
            starting_points = [
                (row, N-1, [], 0, (row, N-1))  # (row, col, path, empty_cell_count)
                for row in range(N - 1)
                if node.board.get((row, N-1)) == 0 or (row, N-1) in current_player_occupied
            ]
            ending_conditions = lambda r, c: (r == N-1 and c != N-1)  # Ends at upper or left wall

        # Initialize the queue
        queue = deque(starting_points)
        visited = set()
        best_path = []
        min_empty_cells = float('inf')
        all_valid_paths = []
        while queue:
            row, col, path, empty_cell_count, origin = queue.popleft()

            if (row, col, origin) in visited:
                continue
            visited.add((row, col, origin))

            # Add the current cell to the path
            new_path = path + [(row, col)]
            #print(new_path)
            new_empty_cell_count = empty_cell_count + (1 if node.board.get((row, col)) == 0 else 0)

            # Stop exploring if empty cell count exceeds the limit
            if new_empty_cell_count > 1:
                continue

            # Check if the path satisfies the ending conditions
            if ending_conditions(row, col) and new_empty_cell_count == 1:
                all_valid_paths.append(new_path)

            # Explore neighbors (horizontal and vertical only)
            neighbors = [
                (row - 1, col), (row + 1, col),  # Vertical
                (row, col - 1), (row, col + 1)   # Horizontal
            ]

            # Filter neighbors to include only valid board coordinates
            neighbors = [
                (nr, nc) for nr, nc in neighbors
                if 0 <= nr < N and 0 <= nc < N  # Ensure within bounds
            ]

            neighbors.sort(key=lambda x: node.board.get(x) == 0)  # Prefer player-occupied cells

            for nr, nc in neighbors:
                if is_valid_cell(nr, nc, visited, new_empty_cell_count):
                    queue.append((nr, nc, new_path, new_empty_cell_count, origin))

        return all_valid_paths

    # Perform the BFS to find the best path
    paths = bfs()
    # Extract and return the empty cells on the best path
    return paths#[cell for path in paths for cell in path if node.board.get(cell) == 0]



In [233]:
def is_valid_region_block(node, path):
    """
    Check if there are opponent squares in the region under a given path,
    if the region has 2 or fewer empty cells, and if there is only 1 cell in the start row.
    The path can have multiple turns, and the region is dynamically calculated.

    Parameters:
    - node: The GameState object.
    - path: A list of tuples representing the cells in the path.

    Returns:
    - True if there are opponent squares in the region or more than 2 empty cells,
      False otherwise.
    """
    N = node.board.N  # Board size (N x N grid)
    current_player = node.my_player
    opponent_squares = (
        node.occupied_squares2 if current_player == 1 else node.occupied_squares1
    )
    invalid_path_set = {(1, 0), (1, 1), (0, 1)}
    if set(path) == invalid_path_set:
        return False
    # Dynamically compute the region under the path
    region = set()
    start_row_count = 0  # Count cells in the start row
    for i, (row, col) in enumerate(path):
        # Add all rows below (for Player 1) or above (for Player 2)
        if current_player == 1:
            for r in range(row, -1, -1):
                region.add((r, col))
            if row == 0:  # Player 1's start row
                start_row_count += 1
        else:
            for r in range(row, N):
                region.add((r, col))
            if row == N - 1:  # Player 2's start row
                start_row_count += 1

    # Check if only 1 cell in the start row
    if start_row_count > 1:
        return False 

    # Count empty cells and check for opponent squares
    empty_cells = 0
    for cell in region:
        if cell in opponent_squares:
            return False  # Opponent square found in the region

    for cell in region:
        if node.board.get(cell) == 0:
            empty_cells += 1
            if empty_cells >= 2:  # Early exit if more than 2 empty cells
                return True

    # If no opponent squares and 2 or more empty cells, return False
    return False


In [234]:
from competitive_sudoku.sudoku import GameState
from joep_player.sudokuai import NodeGameState
import time

In [235]:
def get_player_positions(board_grid):
    """
    Given a grid representing a game board, return the positions occupied by player 1 and player 2.

    Args:
        board_grid (list of list of int): The game board as a 2D list.

    Returns:
        tuple: A tuple of two lists containing the coordinates occupied by player 1 and player 2 respectively.
    """
    player_1_positions = []
    player_2_positions = []

    for row_idx, row in enumerate(board_grid):
        for col_idx, value in enumerate(row):
            if value == 1:
                player_1_positions.append((row_idx, col_idx))
            elif value == 2:
                player_2_positions.append((row_idx, col_idx))

    return player_1_positions, player_2_positions



def test_find_blocking_cell():
    """
    Test the `find_blocking_cell` function with a predefined board setup.
    """

    class MockBoard:
        def __init__(self, grid, n, m):
            self.grid = grid
            self.N = len(grid)  # Board size (N x N grid)
            self.n = n  # Block height
            self.m = m  # Block width

        def get(self, coords):
            row, col = coords
            return self.grid[row][col]

    class MockGameState:
        def __init__(self, board, my_player, occupied_squares1, occupied_squares2):
            self.board = board
            self.my_player = my_player
            self.occupied_squares1 = occupied_squares1
            self.occupied_squares2 = occupied_squares2

    # Create a mock board and game state
    board_grid = [
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 1],
        [1, 1, 0, 0, 1, 0],
        [2, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 2, 1],
    ]
    mock_board = MockBoard(board_grid, n=2, m=3)

    # Define occupied squares for both players
    occupied_squares1, occupied_squares2 = get_player_positions(board_grid)

    # Create the game state
    mock_game_state = GameState(
        board=mock_board,
        current_player=1,
        occupied_squares1=occupied_squares1,
        occupied_squares2=occupied_squares2,
    )
    node = NodeGameState(mock_game_state, my_player=1)

    # Run the function and validate the result
    t = time.time()
    blocking_cell = find_path_player_left(node)
    pp = find_path_player_right(node)
    blocking_cell = blocking_cell+pp
    print([is_valid_region_block(node, path) for path in blocking_cell])
    print(time.time()-t)
    expected_result = 0  # Blocking cell should be (1, 2)

    assert blocking_cell == expected_result, f"Expected {expected_result}, got {blocking_cell}"

    print("Test passed!")

# Run the test case
test_find_blocking_cell()


[True, True, True]
0.0009963512420654297


AssertionError: Expected 0, got [[(3, 0), (3, 1), (2, 1), (2, 2), (1, 2), (0, 2)], [(3, 0), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4), (3, 4), (2, 4), (2, 5)], [(2, 5), (2, 4), (2, 3), (2, 2), (1, 2), (0, 2)]]

In [160]:
for c in range(6, 1,3):
    print(c)